In [ ]:
import sys,os
sys.path.insert(0, os.path.dirname(os.path.realpath('__file__')))
sys.path.insert(1,'../')
from config import conf as cfg
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import plotly.graph_objects as go
from tools import btools
from IPython.display import HTML
from pandarallel import pandarallel # 导入pandaralle

pandarallel.initialize(progress_bar=False)
%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
col_names=['query', 'target', 'qtmscore', 'ttmscore', 'alntmscore', 'fident', 'lddt', 'prob', 'alnlen', 'mismatch', 'gapopen', 'qstart', 'qend', 'tstart', 'tend', 'evalue', 'bits']
df1 = pd.read_csv('/hpcfs/fhome/shizhenkun/codebase/RXNRECer/temp/aa.m8',  sep='\t', index_col=None, names=col_names)
df1 

,query,target,qtmscore,ttmscore,alntmscore,fident,lddt,prob,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits
0,AF-J8D1N4-F1-model_v4,AF-A3Q9F0-F1-model_v4.pdb,0.2453,0.67460,0.67460,0.024,0.4332,0.045,165,95,13,16,165,1,114,0.4360,23
1,AF-J8D1N4-F1-model_v4,AF-Q2GI11-F1-model_v4.pdb,0.2529,0.65650,0.65650,0.048,0.4152,0.051,165,94,17,13,159,3,122,0.4325,24
2,AF-J8D1N4-F1-model_v4,AF-Q5FF38-F1-model_v4.pdb,0.2602,0.64830,0.64830,0.065,0.4093,0.057,167,96,17,11,159,1,125,0.4320,25
3,AF-J8D1N4-F1-model_v4,AF-Q5HCB3-F1-model_v4.pdb,0.2534,0.65510,0.65510,0.052,0.4223,0.051,170,94,15,12,162,1,122,0.4316,24
4,AF-J8D1N4-F1-model_v4,AF-B7L097-F1-model_v4.pdb,0.2620,0.64410,0.64410,0.052,0.4218,0.057,171,99,19,10,160,1,128,0.4297,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2710,AF-J8GG30-F1-model_v4,AF-Q99MS7-F1-model_v4.pdb,0.2232,0.08022,0.08022,0.007,0.6080,0.034,1072,146,10,63,359,784,1712,0.1485,21
2711,AF-J8GG30-F1-model_v4,AF-Q10172-F1-model_v4.pdb,0.2238,0.08187,0.08187,0.004,0.4760,0.034,1438,167,16,14,471,252,1404,0.1475,21
2712,AF-J8GG30-F1-model_v4,ESM-Q06VD6,0.2116,0.08922,0.08922,0.007,0.5748,0.029,1255,144,15,120,366,320,1480,0.1463,20
2713,AF-J8GG30-F1-model_v4,AF-A2AL36-F1-model_v4.pdb,0.2246,0.06501,0.06501,0.006,0.2581,0.034,2305,165,34,22,539,282,2247,0.1398,21


In [ ]:
col_names=['query',  'qtmscore', 'ttmscore', 'alntmscore','target']
df = pd.read_csv('/hpcfs/fhome/shizhenkun/codebase/RXNRECer/temp/aa.m8',  sep='\t', index_col=None, names=col_names)
df 

,query,qtmscore,ttmscore,alntmscore,target
0,AF-J8G8J6-F1-model_v4,0.5640,0.7328,0.7328,AF-A1RW47-F1-model_v4.pdb
1,AF-J8G8J6-F1-model_v4,0.5617,0.7219,0.7219,AF-B3SAE4-F1-model_v4.pdb
2,AF-J8G8J6-F1-model_v4,0.5765,0.6759,0.6759,AF-Q8PW97-F1-model_v4.pdb
3,AF-J8G8J6-F1-model_v4,0.5598,0.5523,0.5523,AF-Q49YA5-F1-model_v4.pdb
4,AF-J8G8J6-F1-model_v4,0.5428,0.5791,0.5791,AF-Q9FAW5-F1-model_v4.pdb
...,...,...,...,...,...
2710,AF-J8D1N4-F1-model_v4,0.2249,0.2537,0.2537,AF-Q645Y8-F1-model_v4.pdb
2711,AF-J8D1N4-F1-model_v4,0.2410,0.2211,0.2211,AF-Q499V3-F1-model_v4.pdb
2712,AF-J8D1N4-F1-model_v4,0.2786,0.1723,0.1723,AF-P44246-F1-model_v4.pdb
2713,AF-J8D1N4-F1-model_v4,0.2935,0.1255,0.1255,AF-Q8C052-F1-model_v4.pdb


In [8]:
df.sort_values(by=['query','alntmscore'], ascending=False)

,query,qtmscore,ttmscore,alntmscore,target
1400,AF-J8HQ06-F1-model_v4,0.3978,0.7808,0.7808,ESM-P15403
1428,AF-J8HQ06-F1-model_v4,0.3778,0.7350,0.7350,ESM-P22169
1541,AF-J8HQ06-F1-model_v4,0.2930,0.7013,0.7013,AF-Q7NVK0-F1-model_v4.pdb
1542,AF-J8HQ06-F1-model_v4,0.2982,0.6952,0.6952,AF-Q48TL1-F1-model_v4.pdb
1547,AF-J8HQ06-F1-model_v4,0.2979,0.6940,0.6940,AF-Q9CHX5-F1-model_v4.pdb
...,...,...,...,...,...
2711,AF-J8D1N4-F1-model_v4,0.2410,0.2211,0.2211,AF-Q499V3-F1-model_v4.pdb
2709,AF-J8D1N4-F1-model_v4,0.2789,0.2147,0.2147,AF-B1HTV8-F1-model_v4.pdb
2712,AF-J8D1N4-F1-model_v4,0.2786,0.1723,0.1723,AF-P44246-F1-model_v4.pdb
2713,AF-J8D1N4-F1-model_v4,0.2935,0.1255,0.1255,AF-Q8C052-F1-model_v4.pdb
